In [ ]:
# --- Install dependencies if not already installed ---
import sys
!{sys.executable} -m pip install qiskit>=0.46.0 qiskit-aer>=0.12.0

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import XGate, ZGate
from qiskit_aer import AerSimulator

# --- Setup Registers ---
q = QuantumRegister(9, 'data')         # 9 data qubits
anc = QuantumRegister(6, 'anc')        # 6 ancilla qubits
c_syndrome = ClassicalRegister(6, 'syndrome')  # For syndrome measurement
c_final = ClassicalRegister(1, 'final')        # For final logical qubit measurement

qc = QuantumCircuit(q, anc, c_syndrome, c_final)
sim = AerSimulator()

# --- Step 1: Encode logical |+> state ---
qc.h(q[0])               # Prepare |+⟩
qc.cx(q[0], q[3])        # Phase-flip protection: replicate
qc.cx(q[0], q[6])
for i in [0, 3, 6]:      # Bit-flip protection within each triplet
    qc.cx(i, i+1)
    qc.cx(i, i+2)
qc.barrier()

# --- Step 2: Inject optional error ---
# Uncomment one to test error correction
#qc.x(q[4])  # Bit-flip in middle of first triplet
#qc.z(q[7])  # Phase-flip in middle of third triplet
#qc.y(q[1])  # Bit + phase flip

qc.barrier()

# --- Step 3: Syndrome measurement (Bit flips) ---

# Triplet 1 (q0-q2)
qc.cx(q[0], anc[0])
qc.cx(q[1], anc[0])
qc.cx(q[2], anc[0])
qc.measure(anc[0], c_syndrome[0])
qc.reset(anc[0])

# Triplet 2 (q3-q5)
qc.cx(q[3], anc[1])
qc.cx(q[4], anc[1])
qc.cx(q[5], anc[1])
qc.measure(anc[1], c_syndrome[1])
qc.reset(anc[1])

# Triplet 3 (q6-q8)
qc.cx(q[6], anc[2])
qc.cx(q[7], anc[2])
qc.cx(q[8], anc[2])
qc.measure(anc[2], c_syndrome[2])
qc.reset(anc[2])

qc.barrier()

# --- Step 4: Syndrome measurement (Phase flips) ---

# q0 vs q3
qc.h(q[0])
qc.h(q[3])
qc.cx(q[0], anc[3])
qc.cx(q[3], anc[3])
qc.measure(anc[3], c_syndrome[3])
qc.reset(anc[3])
qc.h(q[0])
qc.h(q[3])

# q3 vs q6
qc.h(q[3])
qc.h(q[6])
qc.cx(q[3], anc[4])
qc.cx(q[6], anc[4])
qc.measure(anc[4], c_syndrome[4])
qc.reset(anc[4])
qc.h(q[3])
qc.h(q[6])

# q0 vs q6
qc.h(q[0])
qc.h(q[6])
qc.cx(q[0], anc[5])
qc.cx(q[6], anc[5])
qc.measure(anc[5], c_syndrome[5])
qc.reset(anc[5])
qc.h(q[0])
qc.h(q[6])

qc.barrier()

from qiskit.circuit import Gate

# --- Step 5: Conditional Error Correction ---

# Bit-flip corrections
with qc.if_test((c_syndrome, int('100000', 2))):
    qc.x(q[0])

with qc.if_test((c_syndrome, int('010000', 2))):
    qc.x(q[3])

with qc.if_test((c_syndrome, int('001000', 2))):
    qc.x(q[6])

# Phase-flip corrections
with qc.if_test((c_syndrome, int('000100', 2))):
    qc.z(q[0])

with qc.if_test((c_syndrome, int('000010', 2))):
    qc.z(q[3])

with qc.if_test((c_syndrome, int('000001', 2))):
    qc.z(q[6])


# --- Step 6: Decode logical qubit ---
for i in [0, 3, 6]:
    qc.cx(i, i+1)
    qc.cx(i, i+2)

qc.cx(q[0], q[3])
qc.cx(q[0], q[6])
qc.h(q[0])
qc.measure(q[0], c_final[0])

# --- Step 7: Run ---
tqc = transpile(qc, sim)
result = sim.run(tqc, shots=1024).result()
counts = result.get_counts()

# --- Step 8: Parse & Print Results ---

# Format: final_bit syndrome_bitstring (e.g., '0 101000')
syndrome_counts = {}
final_counts = {}

for bitstring, count in counts.items():
    bits = bitstring.replace(' ', '')
    final_bit = bits[0]
    syndrome_bits = bits[1:]

    syndrome_counts[syndrome_bits] = syndrome_counts.get(syndrome_bits, 0) + count
    final_counts[final_bit] = final_counts.get(final_bit, 0) + count

print("Syndrome bitstring counts:")
for s, c in sorted(syndrome_counts.items()):
    print(f"{s}: {c}")

print("\nFinal logical qubit measurement counts (X basis):")
for f, c in sorted(final_counts.items()):
    print(f"{f}: {c}")



Syndrome bitstring counts:
000001: 110
000110: 120
011001: 130
011110: 138
101001: 119
101110: 124
110001: 137
110110: 146

Final logical qubit measurement counts (X basis):
0: 497
1: 527
